In [17]:
import numpy as np
import random
import time
import math
#%run Task.ipynb

In [4]:
class Task:
    def __init__(self, len, processorsRequired):
        self.len = len
        self.p = processorsRequired
        
    def requiers_p1(self):
        return (1 in self.p)
    
    def requiers_p2(self):
        return (2 in self.p)
    
    def requiers_p3(self):
        return (3 in self.p)
    
    def length(self):
        return self.len
    
    def __str__(self):
        return self.len*"|{}|{}|{}|\n".format((' ', 'X')[self.requiers_p1()], (' ', 'X')[self.requiers_p2()], (' ', 'X')[self.requiers_p3()])

In [5]:
class TaskSchedule:
    def __init__(self, original = None):
        self.empty = ' '        # mark for the elements in processors tasks lists that represents time waste
        if original is None:
            self.schedule = np.array([[],[],[]]) # result (only result is store as np array due to efficiency)
            self.task_mark = 0  # mark for the next task that will be added
            self.p1 = []        # processor1 list of tasks
            self.p2 = []        # processor2 list of tasks
            self.p3 = []        # processor3 list of tasks
        else:
            self.schedule = original.schedule
            self.task_mark = original.task_mark  
            self.p1 = original.p1
            self.p2 = original.p2
            self.p3 = original.p3
        
    def __processors_reserved_times(self):
        return [len(self.p1), len(self.p2), len(self.p3)]
        
    def append(self, task):
        # for every processor find time that has been reserved by now
        [p1_len, p2_len, p3_len] = self.__processors_reserved_times() 
        
        # check what is earlier option for task to start
        start = 0
        if (task.requiers_p1()):
            start = p1_len
        if (task.requiers_p2() and start < p2_len):
            start = p2_len
        if (task.requiers_p3() and start < p3_len):
            start = p3_len
        
        # add task to schedule        
        task_in_schedule = task.length() * [self.task_mark]
        if (task.requiers_p1()):
            blank = start - p1_len
            self.p1 = self.p1 + blank * [self.empty]
            self.p1 = self.p1 + task_in_schedule
        if (task.requiers_p2()):
            blank = start - p2_len
            self.p2 = self.p2 + blank * [self.empty]
            self.p2 = self.p2 + task_in_schedule
        if (task.requiers_p3()):
            blank = start - p3_len
            self.p3 = self.p3 + blank * [self.empty]
            self.p3 = self.p3 + task_in_schedule
            
        # change mark for the next task
        self.task_mark = (self.task_mark + 1) % 10
            
    def get_schedule(self):
        [p1_len, p2_len, p3_len] = reserved_time = self.__processors_reserved_times()
        max = np.max(np.array(reserved_time))
        
        self.schedule = np.array([
            self.p1 + (max - p1_len) * [' '], 
            self.p2 + (max - p2_len) * [' '], 
            self.p3 + (max - p3_len) * [' ']
        ])
        return self.schedule
    
    def get_value(self):
        reserved_times = self.__processors_reserved_times()
        
        max = np.max(np.array(reserved_times))
        
        return max

In [47]:
class Backtracking:
    def __init__(self, tasks):
        self.result = TaskSchedule()
        self.initialized = False
        
        start_time = time.time_ns()
        self.__execute(tasks, TaskSchedule())
        end_time = time.time_ns()
        self.duration = end_time - start_time
    
    def __execute(self, tasks, raspored):
        if (tasks.size == 0):
            if raspored.get_value() < self.result.get_value():
                self.result = TaskSchedule(raspored)
            elif not self.initialized:
                self.result = TaskSchedule(raspored)
                self.initialized = True
            return
        for i in range(tasks.size):
            childs_raspored = TaskSchedule(raspored)
            childs_raspored.append(tasks[i])
            childs_tasks = np.delete(tasks, i)
            self.__execute(childs_tasks, childs_raspored)
            
    def get_result(self):
        print(self.result.get_value())
        return self.result.get_schedule()
    
    def get_duration(self):
        return (self.duration / 1000000000)

In [48]:
class Bnb:
    def __init__(self, tasks):
        self.result = TaskSchedule()
        self.initialized = False
        
        start_time = time.time_ns()
        self.__execute(tasks, TaskSchedule())
        end_time = time.time_ns()
        self.duration = end_time - start_time
    
    def __execute(self, tasks, raspored):
        # check if its already greater then our best solution
        if (tasks.size != 0 and self.initialized and self.result.get_value() < raspored.get_value()):
            return
        
        # check if it is leaf of the tree
        if (tasks.size == 0):
            if raspored.get_value() < self.result.get_value():
                self.result = TaskSchedule(raspored)
            elif not self.initialized:
                self.result = TaskSchedule(raspored)
                self.initialized = True
            return
        
        # make all possible options with this root
        for i in range(tasks.size):
            childs_raspored = TaskSchedule(raspored)
            childs_raspored.append(tasks[i])
            childs_tasks = np.delete(tasks, i)
            self.__execute(childs_tasks, childs_raspored)
            
    def get_result(self):
        print(self.result.get_value())
        return self.result.get_schedule()
    
    def get_duration(self):
        return self.duration / 1000000000

In [49]:
class SimulatedAnnealing:
    def __init__(self, tasks):
        self.result = TaskSchedule()
        self.initialized = False
        
        # algorithm parameters #
        self.MAX_TIME_SPENT = 30
        self.MAX_REPETITION = 100
        self.MAX_ITERATION = 100000
        # to set propbability go to 
        # '__update_probability' funciton
        # to set condition for end go to 
        # '__condition_fulfilled' funciton
        # self.c = 100 parametar za apdejt verovatnoce
        ########################
        
        start_time = time.time_ns()
        self.__execute(tasks)
        end_time = time.time_ns()
        self.duration = end_time - start_time
        
    def __create_schedule(self, tasks):
        schedule = TaskSchedule()
        for task in tasks:
            schedule.append(task)
            
        return schedule
    
    def __calculate_fittness(self, tasks):
        schedule = TaskSchedule()
        for task in tasks:
            schedule.append(task)
        
        return schedule.get_value()
    
    def __condition_fulfilled(self, i, repetition):
#        continue1 = i < self.MAX_ITERATION
        continue2 = repetition < self.MAX_REPETITION
        continue3 = time.time() < self.expiring_time

        return continue2 and continue3
    
    # updates the probability for jumping out of local convergence
    def __updateProbability(self, sfit, spfit, i):
    #    p = np.log(2)/np.log(1+i)
        p = 1/np.sqrt(i)
    #    p = np.exp((-sfit-spfit) / self.c)

        return p
    
    def __get_neighbour(self, tasks):
        neighbour_tasks = tasks[:]
        n = len(tasks)
        for i in range(math.floor(n / 10) + 1):
            j = random.randrange(n)
            k = random.randrange(n)
            neighbour_tasks[j], neighbour_tasks[k] = neighbour_tasks[k], neighbour_tasks[j]
        return neighbour_tasks
        
    def __execute(self, tasks):
        # initialize start solution
        s = tasks                 # in order to execute fast we will store only raw Task array (schedule will be generated at the end),
                                  # so that we don't waste time getting that array every time we make neighbour
        s_fit = self.__calculate_fittness(s)
        best_fit = s_fit
        best_s = s

        # set expiring time 
        self.expiring_time = self.MAX_TIME_SPENT + time.time()

        i = 1
        repeat = 0
        while self.__condition_fulfilled(i, repeat):
            sp = self.__get_neighbour(s)
            sp_fit = self.__calculate_fittness(sp)

            if sp_fit < s_fit:
                s = sp
                s_fit = sp_fit
                repeat = 0
            else:
                p = self.__updateProbability(s_fit, sp_fit, i)
                q = random.random()
                if q < p:
                    s = sp
                    s_fit = sp_fit
                    repeat = 0
                else:
                    repeat += 1

            if sp_fit < best_fit:
                best_fit = sp_fit
                best_s = sp

            i+=1

        self.result = self.__create_schedule(best_s)
        
    def get_result(self):
        print(self.result.get_value())
        return self.result.get_schedule()
    
    def get_duration(self):
        return self.duration / 1000000000

In [95]:
class Hibrid:
    def __init__(self, tasks):
        self.result = TaskSchedule()
        self.initialized = False
        
        # algorithm parameters #
        self.MAX_TIME_SPENT = 30
        self.MAX_REPETITION = 50
        self.MAX_ITERATION = 100000
        # to set propbability go to 
        # '__update_probability' funciton
        # to set condition for end go to 
        # '__condition_fulfilled' funciton
        # self.c = 100 parametar za apdejt verovatnoce
        ########################
        
        # first manualy set 3-dedicated tasks to begining of the schedule
        left_tasks = self.__set_begging(tasks)
        
        start_time = time.time_ns()
        self.__execute(left_tasks)
        end_time = time.time_ns()
        self.duration = end_time - start_time
        
    def __set_begging(self, tasks):
        left_tasks = []
        for task in tasks:
            if task.requiers_p1() and task.requiers_p2() and task.requiers_p3():
                self.result.append(task)
            else:
                left_tasks.append(task)

        return left_tasks
    
    def __calculate_fittness(self, tasks):
        schedule = TaskSchedule()
        for task in tasks:
            schedule.append(task)
        
        return schedule.get_value()
    
    def __condition_fulfilled(self, i, repetition):
#        continue1 = i < self.MAX_ITERATION
        continue2 = repetition < self.MAX_REPETITION
        continue3 = time.time() < self.expiring_time

        return continue2 and continue3
    
    # updates the probability for jumping out of local convergence
    def __updateProbability(self, sfit, spfit, i):
    #    p = np.log(2)/np.log(1+i)
        p = 1/np.sqrt(i)
    #    p = np.exp((-sfit-spfit) / self.c)

        return p
    
    def __get_neighbour(self, tasks):
        neighbour_tasks = tasks[:]
        n = len(tasks)
        for i in range(math.ceil(n / 10)):
            j = random.randrange(n)
            k = random.randrange(n)
            neighbour_tasks[j], neighbour_tasks[k] = neighbour_tasks[k], neighbour_tasks[j]
        return neighbour_tasks
        
    def __execute(self, tasks):
        # initialize start solution
        s = tasks                 # in order to execute fast we will store only raw Task array (schedule will be generated at the end),
                                  # so that we don't waste time getting that array every time we make neighbour
        s_fit = self.__calculate_fittness(s)
        best_fit = s_fit
        best_s = s

        # set expiring time 
        self.expiring_time = self.MAX_TIME_SPENT + time.time()

        i = 1
        repeat = 0
        while self.__condition_fulfilled(i, repeat):
            sp = self.__get_neighbour(s)
            sp_fit = self.__calculate_fittness(sp)

            if sp_fit < s_fit:
                s = sp
                s_fit = sp_fit
                repeat = 0
            else:
                p = self.__updateProbability(s_fit, sp_fit, i)
                q = random.random()
                if q < p:
                    s = sp
                    s_fit = sp_fit
                    repeat = 0
                else:
                    repeat = repeat + 1

            if sp_fit < best_fit:
                best_fit = sp_fit
                best_s = sp

            i+=1

        # append best task permutation to schedule
        for task in best_s:
            self.result.append(task)
        
    def get_result(self):
        print(self.result.get_value())
        return self.result.get_schedule()
    
    def get_duration(self):
        return self.duration / 1000000000

In [9]:
class ExampleGenerator:
    def __init__(self):
        self.tasks = np.array([], dtype = Task)
    
    def create_random_tasks(self, num_of_tasks, min_len, max_len):
        # initialize self parameter
        self.num_of_tasks = num_of_tasks
        
        # processors combination: 1,2,3,12,13,23,123
        processors_combination = np.array([
            np.array([1]),
            np.array([2]),
            np.array([3]),
            np.array([1,2]),
            np.array([1,3]),
            np.array([2,3]),
            np.array([1,2,3]),
        ])
        
        tasks_list = []
        for i in range(num_of_tasks):
            # initialize task parameters
            task_len = random.randrange(min_len, max_len+1)
            processors = np.random.choice(processors_combination)
            
            # add task to array
            new_task = Task(task_len, processors)
            tasks_list.append(new_task)
            
        self.tasks = np.array(tasks_list)
    
    def print_tasks(self):
        for i in range(self.num_of_tasks):
            print(self.tasks[i])
            
    def get_tasks(self):
        return self.tasks

In [145]:
# Testing
exmp = ExampleGenerator()
exmp.create_random_tasks(30, 1, 6)
exmp.print_tasks()
taskovi = exmp.get_tasks()

| |X|X|
| |X|X|
| |X|X|

|X| | |
|X| | |

|X|X| |
|X|X| |
|X|X| |
|X|X| |
|X|X| |
|X|X| |

|X|X|X|
|X|X|X|
|X|X|X|
|X|X|X|
|X|X|X|

| | |X|

|X| | |
|X| | |
|X| | |
|X| | |
|X| | |
|X| | |

|X|X| |
|X|X| |
|X|X| |
|X|X| |
|X|X| |
|X|X| |

|X| |X|

|X| | |
|X| | |
|X| | |

| |X|X|
| |X|X|
| |X|X|

| |X| |
| |X| |
| |X| |

| | |X|

|X| | |
|X| | |
|X| | |
|X| | |
|X| | |

| |X| |
| |X| |
| |X| |

|X| | |

| | |X|
| | |X|
| | |X|
| | |X|

|X| |X|
|X| |X|
|X| |X|

|X| |X|

| |X|X|

|X| | |
|X| | |
|X| | |
|X| | |
|X| | |

|X|X|X|
|X|X|X|
|X|X|X|
|X|X|X|

|X|X|X|
|X|X|X|
|X|X|X|
|X|X|X|

|X|X| |
|X|X| |
|X|X| |
|X|X| |

| |X| |
| |X| |
| |X| |

|X|X|X|
|X|X|X|
|X|X|X|
|X|X|X|
|X|X|X|
|X|X|X|

| | |X|
| | |X|
| | |X|
| | |X|

| |X| |
| |X| |
| |X| |
| |X| |
| |X| |

|X| | |

|X| |X|

|X|X|X|
|X|X|X|
|X|X|X|



In [100]:
#bck = Backtracking(taskovi)
#print(bck.get_result())
#bck.print_duration()

print("BNB: strts ...")
bnb = Bnb(taskovi)
print(bnb.get_result())
print("duration:", bnb.get_duration())

BNB: strts ...


KeyboardInterrupt: 

In [147]:
print("SA: starts ... \n")
sa = SimulatedAnnealing(taskovi)
print(sa.get_result())
print("duration:", sa.get_duration())

SA: starts ... 

89
[['0' '1' '1' '1' '1' '1' '1' '2' '3' '3' '3' '3' '3' '4' '5' '6' '6' '6'
  ' ' '8' '8' '8' '8' '8' '9' '9' '0' '0' '0' '0' '0' '2' '2' '2' ' ' ' '
  ' ' ' ' '5' '5' '5' '5' ' ' '7' '7' '7' '7' '8' '8' '8' '9' '9' '9' '9'
  '9' '9' '0' '0' '0' '0' '0' '0' '1' '1' '1' '1' ' ' ' ' ' ' '3' ' ' ' '
  ' ' ' ' '5' '5' '5' '5' '5' '5' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' '1' '1' '1' '1' '1' '1' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '6' '6' '6'
  '7' '8' '8' '8' '8' '8' '3' '3' '3' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' '5' '5' '5' '5' ' ' '7' '7' '7' '7' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' '0' '0' '0' '0' '0' '0' '1' '1' '1' '1' '2' '2' '2' '4' '4' '4'
  '4' '4' '5' '5' '5' '5' '5' '5' '6' '6' '6' '8' '8' '8' '9' '9' '9']
 ['0' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '5' '6' '6' '6'
  '7' '8' '8' '8' '8' '8' '1' ' ' ' ' ' ' ' ' ' ' ' ' '2' '2' '2' '4' '4'
  '4' '4' '5' '5' '5' '5' '6' '7' '7' '7' '7' ' ' ' ' ' ' ' ' ' ' ' ' ' '
  ' ' ' ' '0' '0' '0' '0

In [146]:
print("Hibrid: starts ... \n")
hi = Hibrid(taskovi)
print(hi.get_result())
print("duration:", hi.get_duration())

Hibrid: starts ... 

67
[['0' '0' '0' '0' '0' '1' '1' '1' '1' '2' '2' '2' '2' '3' '3' '3' '3' '3'
  '3' '4' '4' '4' '5' '5' '5' '5' '5' '5' '8' '8' '8' '8' '8' '8' '9' '0'
  '1' '1' '1' '1' '1' '1' '2' '2' '2' '6' '6' '6' '6' '6' '8' '9' '2' '2'
  '2' '2' '4' '5' '5' '6' '6' '6' '9' '9' '9' '9' '9']
 ['0' '0' '0' '0' '0' '1' '1' '1' '1' '2' '2' '2' '2' '3' '3' '3' '3' '3'
  '3' '4' '4' '4' '6' '6' '6' ' ' ' ' ' ' '8' '8' '8' '8' '8' '8' ' ' ' '
  '1' '1' '1' '1' '1' '1' '3' '3' '3' '4' '4' '4' '7' '1' '1' '1' '2' '2'
  '2' '2' '3' '3' '3' '8' '8' '8' '8' '8' ' ' ' ' ' ']
 ['0' '0' '0' '0' '0' '1' '1' '1' '1' '2' '2' '2' '2' '3' '3' '3' '3' '3'
  '3' '4' '4' '4' '6' '6' '6' '7' '7' '7' '7' ' ' ' ' ' ' ' ' ' ' '9' ' '
  ' ' ' ' ' ' ' ' ' ' ' ' '3' '3' '3' '5' ' ' ' ' '7' ' ' ' ' '9' '0' '0'
  '0' '0' '4' ' ' ' ' '6' '6' '6' '7' ' ' ' ' ' ' ' ']]
duration: 1.4929013
